### LSTM model architecture

##### Input x 
Tensor of shape: [batch, seq_len, num_features=3] 
- Given 3D input feature vector

##### Output x: 
Tensor of shape: [batch, 5] 
- Sigmoid probabilities for each finger (0-1)

Hyperparameters to be tuned, but current hyperparameters should be solid.

In [ ]:
class LSTM_model(nn.Module):
    def __init__(self):
        super(LSTM_model, self).__init__()
        self.lstm1 = nn.LSTM(input_size=3, hidden_size=64, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 5) # 5 output for the fingers
    def forward(self, x):
        # Transpose not needed given input tensor shape
        x, _ = self.lstm1(x) 
        x, _ = self.lstm2(x)
        x = x[:, -1, :] # Keep only last timestep (batch, 128)
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        x = torch.sigmoid(x) # Sigmoid activation for each finger
        return x # (batch, 5)